<a href="https://colab.research.google.com/github/Vasiliki655/DSC511-Introduction/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSC511: 2nd Assignment - Map Reduce

In this assignment it is expected of you use the MRJob python library to implement three Map Reduce Tasks. For each task, the input file is specified in the tasks' description.

The deadline is on: <strong>07/03/2025, Friday, @2:00pm</strong><br>
You shall submit your assignment through email:
- Please use as <strong>subject</strong>: "DSC511: 2nd Assignment Submission"
- <strong>Rename your notebook</strong> so to have your name on it: "Assignment2_FirstName_LastName.ipynb"
- The email shall be send to this email: <strong>georgiou.joanna@ucy.ac.cy</strong>
- Do not assume that the submission is completed unless you <strong>have received a reply from your instructor</strong> stating that they got your email. In the case of not receiving an email after 24 hours, please resend your email.
<br><br>

<strong>Grading Key:</strong>
- Each task takes <strong>(28 points)</strong>
- Comments & Quality of Code <strong>(16 points)</strong>
(Regarding the comments, your are expected to put a few simple explanatory comments for your code to explain what it does)

<strong>Always have in mind to:</strong>
- Check the desired output
- Read carefully what is asked of you
- Explore the documents' fields as to be able to perform the tasks succesfully
- Take a glance to the lab's exercises and the lecture's examples.

## Setup


In [1]:
! pip install mrjob

In [2]:
def run_mr_job(mr_job):
  # Create a runner for the MapReduce job
  with mr_job.make_runner() as runner:
      # Run the MapReduce job
      runner.run()

      # Iterate over the output of the MapReduce job
      for key, value in mr_job.parse_output(runner.cat_output()):
          # Print each key-value pair (word, count)
          print(key, value)

## Task 1: Letter Count
Similar to the word count task that we've seen during Lab04, calculate the frequency of each letter across the entire file (meaning, for each character from the English Alphabet, how many times does it appear in the entire file?)<br>

As input, use the [wordcount.txt](https://www.cs.ucy.ac.cy/~jgeorg02/dsc511/hadoop/wordcount.txt). (Download & Upload it to Google Colab, as we learned during the labs)

In [3]:
%%file lettercount.py

from mrjob.job import MRJob

class MRLetterCount(MRJob):

  # Mapper function
  def mapper(self, _, line):
    for word in line.split():
      for letter in word:
        if letter.isalnum():
          yield(letter,1)


  # Reducer function
  def reducer(self, letter, counts):
    yield(letter,sum(counts))

Overwriting lettercount.py


In [4]:
import lettercount

# Uncomment the following two lines when wanting to reload the python file, or you can restart the session
# import importlib
# importlib.reload(lettercount)

mr_job = lettercount.MRLetterCount(args=['wordcount.txt'])
run_mr_job(mr_job)

f 10
g 10
h 18
i 61
k 7
l 41
m 20
n 46
u 16
v 6
w 13
x 1
y 10
o 54
p 15
r 50
s 50
t 57
A 1
F 1
H 1
S 2
T 3
a 67
b 9
c 38
d 33
e 82


## Task 2: Num & Sum of products per country
This time, calculate, for each country, the number of products they have and the sum of their prices. <br>
E.g.
```
Input is: (Cyprus, 100)
          (Cyprus, 200)
Output should be: Cyprus 2 300
```
The input file should be [SalesJan2009.csv](https://www.cs.ucy.ac.cy/~jgeorg02/dsc511/hadoop/SalesJan2009.csv) (Download the file, and upload it as we learned in lab04). <br>
<strong>Hint: </strong> Have in mind that if you strip each line from the dataset, the country attribute is located in column[7] and the price in column[2].

In [5]:
%%file num_products_country.py

from mrjob.job import MRJob

class MRNumProductsCountry(MRJob):

    def mapper(self, _, line):
        # TODO: fill the mapper code
        data=line.split(",")
        country=data[7]
        price=int(data[2])
        key=country
        yield(key,[price,1])

    def reducer(self, key, values):
        # TODO: fill the reducer code
        total_price=0
        count=0
        for value in values:
          total_price+=int(value[0])
          count+=value[1]
        yield(key, f"{count} {int(total_price)}")


Overwriting num_products_country.py


In [6]:
import num_products_country

mr_job = num_products_country.MRNumProductsCountry(args=['SalesJan2009.csv'])
run_mr_job(mr_job)

Moldova 1 1200
Monaco 2 2400
Netherlands 22 44700
New Zealand 6 7200
Norway 16 21600
Philippines 2 2400
Poland 2 2400
Romania 1 1200
Russia 1 3600
South Africa 5 12300
South Korea 1 1200
Spain 12 16800
Sweden 13 22800
Switzerland 36 76800
Thailand 2 4800
The Bahamas 2 2400
Turkey 6 7200
Ukraine 1 1200
United Arab Emirates 6 12000
United Kingdom 100 144000
United States 463 738300
Argentina 1 1200
Australia 38 64800
Austria 7 10800
Bahrain 1 1200
Belgium 8 12000
Bermuda 1 1200
Brazil 5 12300
Bulgaria 1 1200
Canada 76 124800
Cayman Isls 1 1200
China 1 1200
Costa Rica 1 1200
Czech Republic 3 6000
Denmark 15 18000
Dominican Republic 1 1200
Finland 2 2400
France 27 53100
Germany 25 42000
Greece 1 1200
Guatemala 1 1200
Hong Kong 1 1200
Hungary 3 3600
Iceland 1 1200
India 2 2400
Ireland 49 69900
Israel 1 1200
Italy 15 37800
Japan 2 2400
Jersey 1 1200
Kuwait 1 1200
Latvia 1 1200
Luxembourg 1 1200
Malaysia 1 1200
Malta 2 4800
Mauritius 1 3600


## Task 3: Anagram
Last but not least, find the anagrams. <br>
An anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once. <br>
E.g. Refills → fillers, Relayed → layered, Rentals → antlers, Rebuild → builder <br>
As input file use the [unixdict.txt](https://www.cs.ucy.ac.cy/~jgeorg02/dsc511/hadoop/unixdict.txt). The output has the following format: <br>
```python
# 2 is the number of words identified as anagrams,
# following with the list of words
2 ['bella', 'label']
```

In [7]:
%%file anagram.py

from mrjob.job import MRJob

class MRAnagram(MRJob):

    def mapper(self, _, line):
      # TODO: fill the mapper code
      for word in line.split():
        sorted_word="".join(sorted(word))
        yield(sorted_word,word)

    def reducer(self, key, values):
      # TODO: fill the reducer code
      count=0
      l=[]
      for value in values:
        count+=1
        l.append(value)
      yield(count,l)

Overwriting anagram.py


In [8]:
import anagram

mr_job = anagram.MRAnagram(args=['unixdict.txt'])
run_mr_job(mr_job)

1 ['cattlemen']
1 ['someplace']
1 ['emplace']
1 ['opalescent']
1 ['pentecostal']
1 ['screenplay']
2 ['cerulean', 'laurence']
1 ['lawrence']
1 ['cleanse']
1 ['newcastle']
1 ['tentacle']
1 ['nucleate']
1 ['enclave']
1 ['percolate']
1 ['correlate']
1 ['casserole']
1 ['perceptual']
1 ['speculate']
1 ['peculate']
1 ['exculpate']
1 ['cereal']
1 ['electra']
1 ['ulcerate']
1 ['celesta']
1 ['cleave']
1 ['commemorate']
1 ['commensurate']
1 ['menace']
1 ['contemporaneous']
1 ['compensate']
1 ['mercenary']
1 ['cavemen']
1 ['mycenae']
1 ['creamery']
1 ['cremate']
1 ['provenance']
1 ['consternate']
1 ['contravene']
1 ['cantonese']
1 ['penance']
1 ['centenary']
1 ['sustenance']
1 ['canteen']
1 ['cayenne']
1 ['cezanne']
1 ['cotoneaster']
1 ['expectorant']
1 ['nectareous']
1 ['acetone']
1 ['carpenter']
1 ['expectant']
1 ['careen']
1 ['necessary']
1 ['cervantes']
1 ['utterance']
2 ['seance', 'seneca']
1 ['protectorate']
1 ['cooperate']
1 ['procreate']
1 ['watercourse']
1 ['evocate']
1 ['peace']
1 ['pear